In [1]:
import json, hashlib, itertools
from falkordb import FalkorDB

In [5]:
def sid(s):  # stable id from title
    return hashlib.sha1(s.encode()).hexdigest()[:12]

# json extraction
def extract_first_json(text):
    start = text.find('{')
    if start == -1:
        return None
    count = 0
    for i in range(start, len(text)):
        if text[i] == '{':
            count += 1
        elif text[i] == '}':
            count -= 1
            if count == 0:
                return json.loads(text[start:i+1])
    return None

In [14]:
file_path = 'json/2307.16513v2_strict_2p_o3.json'
file_path = 'traces/2307.16513v2_flex_1p_o3.txt'

with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            data = extract_first_json(content)

In [15]:
data

{'nodes': [{'type': 'TASK',
   'name': 'first_order_deception_tasks',
   'canonical_name': 'first_order_deception_tasks',
   'aliases': ['first-order deception task'],
   'confidence': 0.9,
   'notes': 'Binary recommendation and label tasks with burglar scenario'},
  {'type': 'MODEL',
   'name': 'GPT_4',
   'canonical_name': 'GPT-4',
   'aliases': ['gpt-4'],
   'confidence': 0.95,
   'notes': ''},
  {'type': 'RISK_TYPE',
   'name': 'emergent_deception_in_llms',
   'canonical_name': 'emergent_deception_in_llms',
   'aliases': ['deception capability'],
   'confidence': 0.9,
   'notes': ''},
  {'type': 'BENCHMARK',
   'name': 'deception_evaluation_benchmark',
   'canonical_name': 'deception_evaluation_benchmark',
   'aliases': [],
   'confidence': 0.55,
   'notes': 'Inferred safety benchmark to routinely test models'},
  {'type': 'PROMPT_TECHNIQUE',
   'name': 'chain_of_thought_prompting',
   'canonical_name': 'chain_of_thought_prompting',
   'aliases': ['CoT'],
   'confidence': 0.9,
   '

In [22]:
# graph from strict json
file_path = 'traces/2307.16513v2_strict_1p_o3.txt'

with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            data = extract_first_json(content)

# --- 1) build nodes (use title as key + visible name) ---
nodes = []
for n in data["nodes"]:
    label = "Concept" if n.get("type") == "concept" else "Intervention"
    nodes.append({
        "label": label,
        "props": {
            "title": n["title"].strip(),
            "name": n["title"].strip(),               # <- ensures UIs show text, not an ID
            "aliases": n.get("aliases", []),
            "description": n.get("description", ""),
            "maturity": n.get("maturity")
        }
    })

# --- 2) build relationships from logical_chains (match by title) ---
rels = []
for chain in data.get("logical_chains", []):
    for e in chain.get("edges", []):
        retype = e["type"].upper().replace("-", "_")
        rels.append({
            "type": retype,
            "row": {
                "src": e["source_node"].strip(),
                "dst": e["target_node"].strip(),
                "description": e.get("description", ""),
                "confidence": e.get("confidence")
            }
        })

# --- 3) insert into FalkorDB ---
db = FalkorDB(host="localhost", port=6379)
g = db.select_graph("misalignment_v3")

# indexes (safe to re-run)
g.query("CREATE INDEX FOR (c:Concept) ON (c.title)")
g.query("CREATE INDEX FOR (i:Intervention) ON (i.title)")

# nodes (group by label)
by_label = defaultdict(list)
for n in nodes:
    by_label[n["label"]].append(n["props"])

for L, rows in by_label.items():
    for chunk in batched(rows):
        g.query(f"""
        UNWIND $rows AS row
        MERGE (n:`{L}` {{title: row.title}})
        SET n += row
        """, {"rows": chunk})

# relationships (group by type)
by_type = defaultdict(list)
for r in rels:
    # keep only rel props in the payload; src/dst are separate keys
    row = r["row"]
    by_type[r["type"]].append(row)

for T, rows in by_type.items():
    for chunk in batched(rows):
        g.query(f"""
        UNWIND $rows AS row
        MATCH (s {{title: row.src}}), (t {{title: row.dst}})
        MERGE (s)-[r:`{T}`]->(t)
        SET r.description = row.description,
            r.confidence = row.confidence
        """, {"rows": chunk})

print("Done. Nodes keyed by title; 'name' mirrors title for UI display.")

Done. Nodes keyed by title; 'name' mirrors title for UI display.


In [23]:
# graph from flex json
file_path = 'traces/2307.16513v2_flex_1p_o3.txt'

with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            data = extract_first_json(content)

nodes = []
name2id = {}
labels_seen = set()

for n in data["nodes"]:
    key = (n.get("canonical_name") or n["name"]).strip()
    nid = sid(key)
    label = (n["type"] or "ENTITY").upper()   # e.g., TASK, MODEL, PROMPT_TECHNIQUE
    labels_seen.add(label)
    name2id[n["name"]] = nid
    if n.get("canonical_name"): name2id[n["canonical_name"]] = nid  # allow both
    nodes.append({
        "label": label,
        "props": {
            "id": nid,
            "name": n["name"],
            "canonical_name": n.get("canonical_name"),
            "aliases": n.get("aliases", []),
            "confidence": n.get("confidence"),
            "notes": n.get("notes", "")
        }
    })

# --- 2) Relationships from logical_chains ---
rels = []
for chain in data.get("logical_chains", []):
    for e in chain.get("edges", []):
        reltype = (e["type"] or "RELATED").upper().replace("-", "_")
        src = name2id[e["source_node"]]
        dst = name2id[e["target_node"]]
        rels.append({
            "type": reltype,
            "row": {
                "src": src, "dst": dst,
                "rationale": e.get("rationale", ""),
                "confidence": e.get("confidence")
            }
        })

# --- 3) Insert into FalkorDB ---
db = FalkorDB(host="localhost", port=6379)
g = db.select_graph("misalignment_v2")

# indexes (id lookups fast; safe to run repeatedly)
for L in labels_seen:
    g.query(f"CREATE INDEX FOR (n:`{L}`) ON (n.id)")

# nodes (group by label)
by_label = defaultdict(list)
for n in nodes:
    by_label[n["label"]].append(n["props"])

for L, rows in by_label.items():
    for chunk in batched(rows):
        g.query(f"""
        UNWIND $rows AS row
        MERGE (n:`{L}` {{id: row.id}})
        SET n += row
        """, {"rows": chunk})

# relationships (group by type)
by_type = defaultdict(list)
for r in rels:
    by_type[r["type"]].append(r["row"])

for T, rows in by_type.items():
    for chunk in batched(rows):
        g.query(f"""
        UNWIND $rows AS row
        MATCH (s {{id: row.src}}), (t {{id: row.dst}})
        MERGE (s)-[r:`{T}`]->(t)
        SET r += row
        """, {"rows": chunk})

print("Done.")

Done.
